In [1]:
pip install tensorflow pillow matplotlib opencv-python-headless lime


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 803.4 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import InceptionV3, Xception, DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import lime
from lime import lime_image
import cv2
import matplotlib.pyplot as plt

In [19]:
train_dir = "/Users/owaiskhan/Downloads/Lung Disease Dataset/train"
val_dir = "/Users/owaiskhan/Downloads/Lung Disease Dataset/val"
test_dir = "/Users/owaiskhan/Downloads/Lung Disease Dataset/test"


In [21]:
# Model input parameters
IMG_SIZE = (299, 299)
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 31
CLASS_LABELS = ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']


In [23]:
# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_data = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
val_data = val_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
test_data = test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)


Found 6054 images belonging to 5 classes.
Found 2016 images belonging to 5 classes.
Found 2025 images belonging to 5 classes.


In [25]:
# Build and Compile Models
def build_model(base_model):
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(len(CLASS_LABELS), activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [27]:
# Fine-tune Xception and DenseNet121
xception_base = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
xception_model = build_model(xception_base)
xception_model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


In [ ]:
# Train Model with Checkpoints
checkpoint = ModelCheckpoint("best_lung_disease_model.keras", monitor="val_accuracy", save_best_only=True, mode="max")

early_stop = EarlyStopping(monitor="val_accuracy", patience=5)

history = xception_model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stop]
)

# Load Best Model
model = load_model("best_lung_disease_model.h5")


Epoch 1/31


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


190/190 ━━━━━━━━━━━━━━━━━━━━ 5555s 29s/step - accuracy: 0.6394 - loss: 1.1770 - val_accuracy: 0.8398 - val_loss: 0.3817
Epoch 2/31
190/190 ━━━━━━━━━━━━━━━━━━━━ 4996s 26s/step - accuracy: 0.8631 - loss: 0.3651 - val_accuracy: 0.8616 - val_loss: 0.3985
Epoch 3/31
190/190 ━━━━━━━━━━━━━━━━━━━━ 2476s 13s/step - accuracy: 0.8846 - loss: 0.3046 - val_accuracy: 0.8557 - val_loss: 0.3393
Epoch 4/31
190/190 ━━━━━━━━━━━━━━━━━━━━ 2544s 13s/step - accuracy: 0.9034 - loss: 0.2486 - val_accuracy: 0.8780 - val_loss: 0.3166
Epoch 5/31
190/190 ━━━━━━━━━━━━━━━━━━━━ 2239s 12s/step - accuracy: 0.9240 - loss: 0.1950 - val_accuracy: 0.8378 - val_loss: 0.4515
Epoch 6/31
190/190 ━━━━━━━━━━━━━━━━━━━━ 2072s 11s/step - accuracy: 0.9245 - loss: 0.2057 - val_accuracy: 0.8919 - val_loss: 0.2795
Epoch 7/31
190/190 ━━━━━━━━━━━━━━━━━━━━ 2129s 11s/step - accuracy: 0.9417 - loss: 0.1502 - val_accuracy: 0.8790 - val_loss: 0.3213
Epoch 8/31
190/190 ━━━━━━━━━━━━━━━━━━━━ 2089s 11s/step - accuracy: 0.9445 - loss: 0.1533 - val